# Data

In [ ]:
import random
import albumentations as A
import albumentations.augmentations.geometric.functional as F
import cv2

class AspectRatioResize(A.DualTransform):
    """Resize the input to the given height and width.

    Args:
        p (float): probability of applying the transform. Default: 1.
        height (int): desired height of the output.
        width (int): desired width of the output.
        interpolation (OpenCV flag): flag that is used to specify the interpolation algorithm. Should be one of:
            cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4.
            Default: cv2.INTER_LINEAR.

    Targets:
        image, mask, bboxes

    Image types:
        uint8, float32
    """

    def __init__(
        self, height, width, scale=(1, 1), interpolation=cv2.INTER_LINEAR, always_apply=False, p=1
    ):
        super(AspectRatioResize, self).__init__(always_apply, p)
        self.height = height
        self.width = width
        self.scale = scale
        self.interpolation = interpolation

    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        height, width, _ = img.shape
        r = min(self.width / width, self.height / height)
        scale = random.uniform(*self.scale)
        return F.resize(
            img, height=int(scale * r * height), width=int(scale * r * width),
            interpolation=interpolation
        )

    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox

In [ ]:
import albumentations as A

# IMAGE_SIZE = (416, 416)
IMAGE_SIZE = (1088, 608)
# IMAGE_SIZE = (608, 608)
train_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(0.2, 1.0), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT, #BORDER_WRAP,
            value=[128, 128, 128]
        ),
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=100,
        min_visibility=0,
        label_fields=["labels"]
    )
)

val_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(1.0, 1.0), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["labels"]
    )
)

In [ ]:
GENDERS = ["M", "F"]
AGES = ["1-2", "3-9", "10-20", "21-25", "26-27", "28-31", "32-36", "37-45", "46-54", "55-65", "66-116"]
LABELS = GENDERS

## Crowd human

In [ ]:
from pathlib import Path

data_crowd_dir = Path("/home/username/Downloads/crowdhuman")

In [ ]:
import pandas as pd

def read_df(data_file):
    data_df = pd.read_csv(data_file)
    # data_df = data_df[(data_df["head_attr.ignore"] != 1) & (data_df["head_attr.occ"] != 1)]
    data_df = data_df.rename(columns={"gender": "label"})
    # data_df = data_df.rename(columns={"age": "label"})
    data_df = data_df[data_df["label"].notnull()] # & data_df["age"].notnull()]
    data_df = data_df.reset_index(drop=True)
    data_df.loc[pd.notnull(data_df["label"]), "label"] = (
        data_df.loc[pd.notnull(data_df["label"]), "label"].apply(lambda x: LABELS.index(x))
        )
    
    return data_df

In [ ]:
import pandas as pd

# train_crowd_file = Path("../../../mlops/crowdhuman/annotation_train_with_classes.csv")
train_crowd_file = Path("../../../mlops/crowdhuman/correction/checkpoint_annotations.csv")
train_crowd_df = read_df(train_crowd_file)
train_crowd_df = train_crowd_df.rename(columns={"image_name": "id"})
train_crowd_df.head(1)

In [ ]:
train_crowd_df["label"].value_counts()

In [ ]:
train_crowd_df.shape

In [ ]:
val_crowd_file = Path("../../../mlops/crowdhuman/annotation_val_with_classes.csv")
val_crowd_df = read_df(val_crowd_file)
val_crowd_df.head(1)

In [ ]:
val_crowd_df.shape

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

train_crowd_dir = Path(data_crowd_dir, "Images")
train_crowd_dataset = TrainDataset(train_crowd_dir, train_crowd_df, train_transform)
val_crowd_dataset = TrainDataset(train_crowd_dir, val_crowd_df, val_transform)
len(train_crowd_dataset), len(val_crowd_dataset)

In [ ]:
# train_crowd_dataset.data_groups = train_crowd_dataset.data_groups[:1000]
# val_crowd_dataset.data_groups = val_crowd_dataset.data_groups[:1000]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import cv2

image, centers, offsets, labels = train_crowd_dataset[44].values()
for center, offset, label in zip(centers, offsets, labels):
    x, y = map(int, center)
    l, t, r, b = map(int, offset)
    cv2.rectangle(
        image,
        (x - l, y - t),
        (x + r, y + b),
        (1.0, 0.0, 0.0) if label == 1 else (0.0, 0.0, 1.0) if label == 0 else (1.0, 1.0, 0.0),
        2
    )
#     if not np.isnan(label):
#         cv2.putText(
#             image, LABELS[int(label)], (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (1, 1, 1), 2
#         )

plt.figure()
plt.imshow(image)
plt.show()

## Khaliji

In [ ]:
from pathlib import Path

data_khaliji_dir = Path("../../../mlops/khaliji")
!ls $data_khaliji_dir

In [ ]:
import pandas as pd

train_khaliji_file = Path("../../../mlops/khaliji/annotation_train_with_classes.csv")
train_khaliji_df = read_df(train_khaliji_file)
train_khaliji_df = train_khaliji_df.rename(columns={"image_name": "id"})
train_khaliji_df.head(1)

In [ ]:
train_khaliji_df["label"].value_counts()

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

train_khaliji_dir = Path(data_khaliji_dir, "Images")
train_khaliji_dataset = TrainDataset(train_khaliji_dir, train_khaliji_df, train_transform)
len(train_khaliji_dataset)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import cv2

image, centers, offsets, labels = train_khaliji_dataset[1].values()
for center, offset, label in zip(centers, offsets, labels):
    x, y = map(int, center)
    l, t, r, b = map(int, offset)
    cv2.rectangle(
        image,
        (x - l, y - t),
        (x + r, y + b),
        (1.0, 0.0, 0.0) if label == 1 else (0.0, 0.0, 1.0) if label == 0 else (1.0, 1.0, 0.0),
        2
    )
#     if not np.isnan(label):
#         cv2.putText(
#             image, LABELS[int(label)], (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (1, 1, 1), 2
#         )

plt.figure()
plt.imshow(image)
plt.show()

# Modeling

## Training

tensorboard --logdir=log --host=0.0.0.0 --port=8099

In [ ]:
import os
import torch

def seedEverything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
!mkdir -p checkpoint log

In [ ]:
import sys
sys.path.append("../..")
from person_detection.centernet.src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2, "id": 128, "clss": len(LABELS)}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
# model_path = Path("/home/username/Downloads/crowdhuman_dla34.pth)"
model_path = Path("/home/username/Downloads/fairmot_dla34.pth")
# model_path = Path("checkpoint/14.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda", checkpoint_dir="checkpoint", log_dir="log",
    model_path=None
)

In [ ]:
trainer.optimizer.param_groups[0]["lr"]

In [ ]:
train_dataset = torch.utils.data.ConcatDataset([train_crowd_dataset, train_khaliji_dataset])

In [ ]:
seedEverything(2020)
trainer.fit(train_dataset, val_crowd_dataset, batch_size=16, start_epoch=0, end_epoch=20)

In [ ]:
trainer.optimizer.param_groups[0]["lr"] = 1e-5
trainer.optimizer.param_groups[0]["weight_decay"] = 0

In [ ]:
seedEverything(2020)
trainer.fit(train_dataset, val_crowd_dataset, batch_size=16, start_epoch=20, end_epoch=30)

In [ ]:
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda", checkpoint_dir="checkpoint", log_dir="log",
    model_path="checkpoint/18.pth"
)

In [ ]:
trainer.eval(val_crowd_dataset, batch_size=16)

## Pseudo labeling

In [ ]:
import pandas as pd

def read_null_df(data_file):
    data_df = pd.read_csv(data_file)
    data_df = data_df[(data_df["head_attr.ignore"] != 1) & (data_df["head_attr.occ"] != 1)]
    data_df = data_df[data_df["gender"].isnull() & data_df["age"].isnull()]
    data_df = data_df.reset_index(drop=True)
    return data_df

In [ ]:
train_crowd_file = Path("../../../mlops/crowdhuman/annotation_train_with_classes.csv")
train_null_df = read_null_df(train_crowd_file)
train_null_df.head(1)

In [ ]:
import albumentations as A

# IMAGE_SIZE = (416, 416)
IMAGE_SIZE = (1088, 608)
# IMAGE_SIZE = (608, 608)

test_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(1, 1), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=["bboxes_id"]
    )
)

In [ ]:
from src.test_dataset import TestDataset
%load_ext autoreload
%autoreload 2

train_dir = Path(data_dir, "Images")
test_dataset = TestDataset(train_dir, train_null_df, test_transform)

In [ ]:
len(test_dataset), train_null_df.shape

In [ ]:
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda", checkpoint_dir="checkpoint",
    model_path="checkpoint/10.pth"
)

In [ ]:
test_df = trainer.predict(test_dataset, batch_size=16)

In [ ]:
test_df.shape, train_null_df.shape

In [ ]:
test_df["pred"].value_counts()

In [ ]:
new_train_null_df = pd.merge(train_null_df, test_df, on=["id", "extra.box_id"], how="inner")
new_train_null_df = new_train_null_df.loc[
    (new_train_null_df["score"] > 0.9) & (new_train_null_df["score"] < 0.95),
    ["id", "hbox", "vbox", "fbox", "pred"]
]
new_train_null_df = new_train_null_df.rename(columns={"pred": "gender"})
# new_train_null_df = new_train_null_df.rename(columns={"pred": "age"})
new_train_null_df = new_train_null_df.reset_index(drop=True)
new_train_null_df.head(1)

In [ ]:
new_train_null_df["gender"].value_counts()

In [ ]:
new_train_null_df.shape,

In [ ]:
from torch.utils.data import ConcatDataset

train_null_dataset = TrainDataset(train_dir, new_train_null_df, train_transform)
new_train_dataset = ConcatDataset([train_dataset, train_null_dataset])

In [ ]:
nb = new_train_null_df["gender"].value_counts() + train_crowd_df["gender"].value_counts()
nb = nb.sort_index().values

In [ ]:
nb.sum() / (nb * len(nb))

In [ ]:
import torch
trainer.criterion.criterion.weight = torch.as_tensor(
    nb.sum() / (nb * len(nb)), dtype=torch.float32, device=trainer.device
)

In [ ]:
seedEverything(2020)
trainer.fit(new_train_dataset, val_crowd_dataset, batch_size=16, epochs=20)